In [7]:
import requests
import hmac
import json
import base64
import hashlib
import pandas as pd
import time
from time import sleep
import datetime as datetime

url = 'http://www.okex.com'
api_key = 'a3189b93-de8d-4b06-bb5e-76c56f3d55f6'
SecretKey = '45D46FDACB270D64F05ADE3B479A8878'

# signature
def signature(timestamp, method, request_path, body,SecretKey):
    if str(body) == '{}' or str(body) == 'None':
        body = ''
    message = str(timestamp) + str.upper(method) + request_path + str(body)
    mac = hmac.new(bytes(SecretKey, encoding='utf8'), bytes(message, encoding='utf-8'), digestmod='sha256')
    d = mac.digest()
    return base64.b64encode(d)


# set request header
def get_header(body, endpoint, request):
    header = dict()
    header['CONTENT-TYPE'] = 'application/json'
    header['OK-ACCESS-KEY'] = api_key
    header['OK-ACCESS-SIGN'] = signature(time, request, endpoint , body, SecretKey)
    header['OK-ACCESS-TIMESTAMP'] = time
    header['OK-ACCESS-PASSPHRASE'] ='algotrading'
    return header

In [ ]:
while True:
    try:
        eth_old = requests.get(url+'/api/v5/market/ticker?instId=ETH-USD-SWAP').json()
        print(eth_old['data'][0]['bidPx'])
    except exception as e:
        print('Unable to obtain price data')
    
    sleep(300)
    
    try:
        eth_new = requests.get(url+'/api/v5/market/ticker?instId=ETH-USD-SWAP').json()
        print(eth_new['data'][0]['bidPx'])
    except exception as e:
        print('Unable to obtain price data')
        
    percent = float(((float(eth_new['data'][0]['bidPx']) - float(eth_old['data'][0]['bidPx'])) * 100) / float(eth_old['data'][0]['bidPx']))
    
    if percent >= 5:
        
        try:
            body= {"instId" : 'BTC-USD-SWAP',
            "side": "buy",
            "ordType" : "market",
            "sz": "1",}

            endpoint= '/api/v5/trade/order'
            request= 'POST'
            t = datetime.datetime.now()
            time =  str(int(time.mktime(t.timetuple())*1000))
            header = get_header(body=body, endpoint=endpoint, request=request)
            response = requests.post(url,
                                    data=None,
                                    headers=header)
            response.json()
            ordId = response['data'][0]['ordId']
        except exception as e:
            print('Order not launched')
            
        sleep(2)
        
        try:
            body= {'instId':'BTC-USD-SWAP',
                   'ordId': ordId}

            endpoint= '/api/v5/trade/order'
            request= 'GET'
            t = datetime.datetime.now()
            time =  str(int(time.mktime(t.timetuple())*1000))
            header = get_header(body=body, endpoint=endpoint, request=request)
            response = requests.get(url,
                                    data=None,
                                    headers=header)
            response.json()
            check = response['data'][0]['state']
        except exception as e:
            print('Unable to obtain order details')
            
        if check == 'live' or 'filled':
            print('Order launched')
        else:
            print('Order not live or filled')
            
    else:
        print('Requirement not matched')
        sleep(60)